In [ ]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt2
import pandas as pd

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
import tflib.plot


Lambda = 10.0
BATCH_SIZE = 30
BATCH_SIZE_2 = 100
dimensionality = 13
num_epochs = 100
num_iters = 1000
k=2
max_grad_norm = 1000
gamma = 1.0

# main parameters

rho = 0.5
C=10.0

# additional parameters
sigma=0.8

number_of_neurons_regr = 100
number_of_points = 303-33
number_of_neurons = number_of_points

In [ ]:
heart = pd.read_csv('/home/rust/Desktop/SDR/heart.csv')
X = heart.iloc[:,:-1].values
y = heart.iloc[:,13].values
print(len(y))
data = []
target = []
rr = np.random.permutation(303)
for x in rr:
    data.append(X[x])
    target.append(y[x])
# training data
_train_x = np.array(data)
_train_y = np.array(target)
print(_train_x.shape)
print(_train_y.shape)

In [ ]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
Dataplace = tf.placeholder(tf.float32, shape=(dimensionality, number_of_points))
Data = tf.placeholder(tf.float32, shape=(BATCH_SIZE, dimensionality))
outputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE,1)) 

tf_data_x = gamma*tf.random_normal([BATCH_SIZE, dimensionality]) # аргументы функции
tf_data_y = tf.reduce_mean(tf.math.cos(tf.matmul(tf_data_x, Dataplace)), axis=1) # значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_w_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr,1))
tf_B_regr = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons_regr))
tf_bias_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr))


tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

# characteristic function parameters
w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.0), name="neuron_weights")
B = tf.Variable(initial_value=Dataplace, name="weights")
# regression function parameters
w_regr = tf.Variable(tf.random_normal([number_of_neurons_regr,1], stddev=0.35), name="neuron_weights")
B_regr = tf.Variable(tf.random_normal([dimensionality,number_of_neurons_regr], stddev=0.35), name="weights")
bias_regr = tf.Variable(tf.random_normal([number_of_neurons_regr], stddev=0.0), name="biases")

prediction = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_x, B)), tf.nn.sigmoid(w))
penalty = tf.square((2/number_of_neurons)*tf.reduce_sum(tf.nn.sigmoid(w))-1.0)

out_loss = (1-rho)*tf.reduce_mean(tf.square(prediction - tf_data_y)) + C*penalty

prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, B)), tf.nn.sigmoid(w))
prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, B_regr), bias_regr)), w_regr)

grad_psi = tf.reshape(tf.gradients(prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
grad_psi_regr = tf.reshape(tf.gradients(prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

tf_prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, tf_data_B)), tf.nn.sigmoid(tf_data_w))
tf_prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, tf_B_regr), tf_bias_regr)), tf_w_regr)

tf_data_grad_psi = tf.reshape(tf.gradients(tf_prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
tf_data_grad_psi_regr = tf.reshape(tf.gradients(tf_prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)
old_part_regr = tf.matmul(tf_data_grad_psi_regr, tf_data_P)

loss = out_loss + Lbd*(1-rho)*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi, old_part)), axis=1)) + Lbd*rho*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi_regr, old_part_regr)), axis=1))

#offset = tf.random.uniform(shape=[], minval=0, maxval=12, dtype=tf.int32)*BATCH_SIZE
x_plus_error = Data+sigma*tf.random_normal([BATCH_SIZE, dimensionality])
regression = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(x_plus_error, B_regr), bias_regr)), w_regr)
sdr = -tf.reduce_mean(tf.multiply(regression, outputs)) + tf.reduce_mean(tf.math.log(1.0+tf.math.exp(regression)))

loss_hybrid = loss + rho*sdr

In [ ]:
def euclidean_distance(pnt1, pnt2):
    '''Finds the distance between 2 points: pnt1, pnt2'''
    # element-wise computations are automatically handled by numpy
    return sum((pnt1 - pnt2) ** 2)

from collections import defaultdict

def find_majority(labels):
    '''Finds the majority class/label out of the given labels'''
    # defaultdict(type) is to automatically add new keys without throwing error.
    counter = defaultdict(int)
    for label in labels:
        counter[label] += 1

    # Finding the majority class.
    majority_count = max(counter.values())
    for key, value in counter.items():
        if value == majority_count:
            return key

def new_predict(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

def new_predict_regr(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return np.mean(k_labels)

In [ ]:
target = tf.train.AdamOptimizer(learning_rate=3e-5, beta1=0.5, beta2=0.9).minimize(loss_hybrid)

In [ ]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [ ]:
scores = []
scores3 = []
for part in range(9):
    print("Current part is %d\n" % part)
    train_x = np.concatenate((_train_x[0:(part*33)], _train_x[(part+1)*33:303]), axis=0)
    train_y = np.concatenate((_train_y[0:(part*33)], _train_y[(part+1)*33:303]), axis=0)
    test_x = _train_x[part*33:(part+1)*33]
    test_y = _train_y[part*33:(part+1)*33]
    scaler = preprocessing.StandardScaler().fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    transpose_train_x = np.transpose(train_x)

    cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
    cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
    cur_biases = np.zeros((number_of_neurons))
    cur_P = np.zeros((dimensionality, dimensionality)) 
    cur_w_regr = np.random.normal(0, 0.35, (number_of_neurons_regr,1))
    cur_B_regr = np.random.normal(0, 0.35, (dimensionality, number_of_neurons_regr))
    cur_bias_regr = np.random.normal(0, 0.35, (number_of_neurons_regr))
    
    O = np.zeros((dimensionality, k)) 
    cur_iter = 0
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init, feed_dict={Dataplace: transpose_train_x})
    
    for epoch in range(num_epochs):
        print("Epoch %d" %(epoch))
        for iteration in range(num_iters):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            sess.run(target, feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x,
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
        reses = []
        for i in range(100):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            res = sess.run([loss_hybrid], feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x, 
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
            reses.append(res)
        print ("Iter %d: loss: %.4f\n" %(cur_iter, np.mean(np.array(reses))))
        lib.plot.plot('train cost', np.mean(np.array(reses)))

        cur_w, cur_B, cur_w_regr, cur_B_regr, cur_bias_regr = sess.run([w, B, w_regr, B_regr, bias_regr])

        third_grad_psi = np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))
        third_grad_psi_regr = np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))
        for r in range(1000):
            sess.run([tf_data_second])
            np.concatenate((third_grad_psi, np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))), axis=0) 
            np.concatenate((third_grad_psi_regr, np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))), axis=0)

        M = (1-rho)*np.matmul(np.transpose(third_grad_psi), third_grad_psi)+\
            rho*np.matmul(np.transpose(third_grad_psi_regr), third_grad_psi_regr)
        u, s, vh = np.linalg.svd(M, full_matrices=True)
        O = u[:,0:k:1]
        print(s)
        cur_P = np.matmul(O, np.transpose(O))
        tvr = 1-np.sum(np.multiply(s[0:k],s[0:k]))/np.sum(np.multiply(s,s))
        lib.plot.plot('total variance to retain', tvr)
        lib.plot.tick()
        lib.plot.flush()
    reduced_train_x = np.matmul(train_x, O)
    reduced_test_x = np.matmul(test_x, O)
    clf = LogisticRegression(random_state=0).fit(reduced_train_x, train_y)
    score = clf.score(reduced_test_x, test_y)
    scores.append(score)
    print ("Part %d: rate on test %.4f\n" %(part, score))
    test_rgb = []
    for h in test_y:
        if h == 1:
            test_rgb.append('r')
        else:
            test_rgb.append('b')
    plt2.close()
    plt2.scatter(np.transpose(reduced_test_x)[0], np.transpose(reduced_test_x)[1], alpha=0.2, c=np.array(test_rgb))
    plt2.xlabel("1st component")
    plt2.ylabel("2nd component")
    plt2.show()
    sess.close()
    print(cur_P)
    
    i = 0
    total_correct = 0
    for test_image in reduced_test_x:
        pred = new_predict(10, reduced_train_x, train_y, test_image)
        if pred == test_y[i]:
            total_correct += 1
        score3 = (total_correct / (i+1)) * 100            
        i += 1
    print('acc:', str(round(score3, 2))+'%')
    scores3.append(score3)

print ("Average rate on test %.4f\n" %(np.mean(np.array(scores))))
print ("Average 10-NN acc on test %.4f\n" %(np.mean(np.array(scores3))))